# Spot finding pipeline - physics-based, on GPU

Pipeline to find spots, using Peter Brown's functions for physics priors, using GPU, and running on several tiles in parallel.
The global plan is:
  - [x] apply the pipeline to a single tile, non skewed
    - [x] extract single ct xyz tile with spots
  - [ ] handle inputs to use pixel guesses from users instead of physics inputs
  - [ ] add support for skewed tiles
    - [ ] extract tilted tile
  - [ ] adapt napari-spot-detection plugin to this pipeline


In [1]:

import numpy as np
import matplotlib.pyplot as plt
import napari
import scipy.signal
import scipy.ndimage

from pathlib import Path
import warnings
import time
import os
import sys
import joblib
import gc

from tifffile import tifffile
import zarr
import dask.array as da
from dask_image.imread import imread
from dask import delayed
from skimage.io.collection import alphanumeric_key
from pycromanager import Dataset
from napari.qt.threading import thread_worker
# from magicgui import magicgui
# from matplotlib.colors import PowerNorm, LinearSegmentedColormap, Normalize
# from tiler import Tiler
# from tysserand import tysserand as ty
# from mosna import mosna as mo

import localize_psf.rois as roi_fns
from localize_psf import fit
from localize_psf import data_io
import localize_psf.fit_psf as psf
from localize_psf import localize
import image_post_processing as ipp
from image_post_processing import deskew

## On single cartesian volume

### Extract data

In [2]:
dir_load = Path('../../Data/20210825a_sina_widefield')
round = 0
channel = 2
tile = 1

path_metadata = dir_load / f'r000{round}_scan_metadata.csv'
path_im = dir_load / f'sina_10gene_panel_r000{round}_xyz000{tile}_1'
dataset = Dataset(str(path_im))

Dataset opened                 


In [3]:
metadata = data_io.read_metadata(path_metadata)
metadata

{'': 0,
 'root_name': 'sina_10gene_panel',
 'scan_type': 'iterative-widefield',
 'theta': '',
 'scan_step_um': '',
 'pixel_size_um': 0.065,
 'axial_step_um': 0.25,
 'y_pixels': 2048,
 'x_pixels': 2048,
 'exposure_ms': 500.0,
 'camera': 'BSI_Express',
 'readout_mode': '11bit_sensitive',
 'num_r': 8,
 'num_y': 49,
 'num_z': 121,
 'num_ch': 3,
 'scan_axis_positions': '',
 '405_active': False,
 '488_active': True,
 '561_active': True,
 '635_active': True,
 '730_active': False,
 '405_power': 0.0,
 '488_power': 1.0,
 '561_power': 49.6098,
 '635_power': 50.0,
 '730_power': 0.0}

In [4]:
# ###############################
# load/set scan parameters
# ###############################
metadata = data_io.read_metadata(path_metadata)

scan_type = metadata["scan_type"]
if scan_type == 'iterative-widefield':
    nt = metadata["num_r"]
    nyp = metadata["y_pixels"]
    nxp = metadata["x_pixels"]
    dc = metadata['pixel_size_um']
    dstage = metadata['axial_step_um']
    # do we need this for later analyses?:
    # theta = 90 * np.pi / 180
    # normal = np.array([0, -np.sin(theta), np.cos(theta)])  # normal of camera pixel

    num_r = metadata['num_r']
    num_y = metadata['num_y']
    num_z = metadata['num_z']
    num_ch = metadata['num_ch']
    nimgs_per_vol = num_z  # check that

    # trapezoid volume
    volume_um3 = (dstage * nimgs_per_vol) * (dc * nyp) * (dc * nxp)
else:
    nt = metadata["num_t"]
    nyp = metadata["y_pixels"]
    nxp = metadata["x_pixels"]
    dc = metadata["pixel_size"] / 1000
    dstage = metadata["scan_step"] / 1000
    theta = metadata["theta"] * np.pi / 180
    normal = np.array([0, -np.sin(theta), np.cos(theta)])  # normal of camera pixel

    num_r = metadata['num_r']
    num_y = metadata['num_y']
    num_z = metadata['num_z']
    num_ch = metadata['num_ch']
    num_images = metadata['scan_axis_positions']
    excess_images = metadata['excess_scan_positions']
    nimgs_per_vol = num_images + excess_images

    # trapezoid volume
    volume_um3 = (dstage * nimgs_per_vol) * (dc * np.sin(theta) * nyp) * (dc * nxp)

In [5]:
# ###############################
# load/set parameters for all datasets
# ###############################
chunk_size_planes = 1000
chunk_size_x = 300
# chunk_size_planes = 300
# chunk_size_x = 150
chunk_overlap = 10
channel_to_use = [False, True, True]
excitation_wavelengths = np.array([0.488, 0.561, 0.635])
emission_wavelengths = np.array([0.515, 0.600, 0.680])
thresholds = np.array([np.nan, 10, 10])
fit_thresholds = np.array([np.nan, 10, 10])
na = 1.
ni = 1.4

In [6]:
# TODO: make genral reader to handle tiff stacks, NDTiffs and zarr arrays given scan parameters
ch = 2

# load all images
imgs = data_io.open_NDTiff(dataset, channels=ch+1)
imgs = imgs[51:74, 449:800, 1000:1400]
# imgs = img[51:88, 49:756, 746:1409]
imgs = np.flip(np.asarray(imgs), axis=0)

    channel id: 3


In [7]:
viewer = napari.Viewer()
viewer.add_image(
    imgs, 
    # contrast_limits=[mini, maxi], 
    # name='ch_' + str(channel), 
    # # colormap=color, 
    blending='additive',
    )

/home/alexis/.pyenv/versions/3.8.10/envs/improcess/lib/python3.8/site-packages/napari_tools_menu/__init__.py:168: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Image layer 'imgs' at 0x7fb265dc80a0>

in _make_sigmas:
na 1.0
ni 1.4
lambda_em 0.68
self.sigma_z 0.7422713547744596
self.sigma_xy 0.1619991731959446
-------
in _make_roi_sizes
dz 0.25
dxy 0.065
self.fit_roi_sizes [3 1 1]
self.min_roi_sizes_pix [6.5 8.5 8.5]
------------
in _make_min_spot_sep:
sigma_z 0.7422713547744596
sigma_xy 0.1619991731959446
sigmas_min (0.1855678386936149, 0.04049979329898615)
sigmas_max (2.2268140643233787, 0.6479966927837784)
filter_sigma_small (0.3711356773872298, 0.04049979329898615, 0.04049979329898615)
filter_sigma_large (2.2268140643233787, 0.48599751958783377, 0.48599751958783377)
fit_roi_sizes [3 1 1]
min_spot_sep (2.2268140643233787, 0.48599751958783377)
-------


In [8]:
imgs.shape

(23, 351, 400)

In [9]:
# reload(localize)
# image coordinates
# x, y, z = localize.get_coords(imgs.shape, dc, dstage, theta)
z, y, x = localize.get_coords(imgs.shape, [dstage, dc, dc])
zb, yb, xb = np.broadcast_arrays(z, y, x)

In [10]:
# ###############################
# identify candidate points in opm data
# ###############################
# sigma_xy = 0.22 * emission_wavelengths[ch] / na
# sigma_z = np.sqrt(6) / np.pi * ni * emission_wavelengths[ch] / na ** 2
sigma_xy = psf.na2sxy(na, emission_wavelengths[ch])
sigma_z = psf.na2sz(na, emission_wavelengths[ch], ni)

# difference of gaussian filer
filter_sigma_small = (0.5 * sigma_z, 0.25 * sigma_xy, 0.25 * sigma_xy)
filter_sigma_large = (3 * sigma_z, 3 * sigma_xy, 3 * sigma_xy)
# fit roi size
roi_size = (5 * sigma_z, 12 * sigma_xy, 12 * sigma_xy)
# assume points closer together than this come from a single bead
min_spot_sep = (3 * sigma_z, 3 * sigma_xy)
# exclude points with sigmas outside these ranges
sigmas_min = (0.25 * sigma_z, 0.25 * sigma_xy)
sigmas_max = (3 * sigma_z, 4 * sigma_xy)

In [11]:
print('ni', ni)
print('na', na)
print('lambda_em', emission_wavelengths[ch])
print('sigma_z', sigma_z)
print('sigma_xy', sigma_xy)
print('sigmas_min', sigmas_min)
print('sigmas_max', sigmas_max)
print('filter_sigma_small', filter_sigma_small)
print('filter_sigma_large', filter_sigma_large)
# print('filter_sigma_small', fit_roi_sizes)
# print('filter_sigma_small', min_fit_roi_sizes)
# print('filter_sigma_large', min_roi_sizes_pix)
print('min_spot_sep', min_spot_sep)

ni 1.4
na 1.0
lambda_em 0.68
sigma_z 0.7422713547744596
sigma_xy 0.1619991731959446
sigmas_min (0.1855678386936149, 0.04049979329898615)
sigmas_max (2.2268140643233787, 0.6479966927837784)
filter_sigma_small (0.3711356773872298, 0.04049979329898615, 0.04049979329898615)
filter_sigma_large (2.2268140643233787, 0.48599751958783377, 0.48599751958783377)
min_spot_sep (2.2268140643233787, 0.48599751958783377)


In [12]:
# do localization
# ###################################################
# smooth image and remove background with difference of gaussians filter
# ###################################################
imgs_chunk = imgs

drs = [dstage, dc, dc]
ks = localize.get_filter_kernel(filter_sigma_small, drs, sigma_cutoff=2)
kl = localize.get_filter_kernel(filter_sigma_large, drs, sigma_cutoff=2)
imgs_hp = localize.filter_convolve(imgs_chunk, ks, use_gpu=False)
imgs_lp = localize.filter_convolve(imgs_chunk, kl, use_gpu=False)
imgs_filtered = imgs_hp - imgs_lp

In [13]:
# viewer = napari.Viewer()
# viewer.add_image(imgs, name='imgs', blending='additive')
viewer.add_image(imgs_filtered, name='imgs_filtered', blending='additive')

<Image layer 'imgs_filtered' at 0x7f1e5dcd2550>

In [16]:
# here we don't handle chuncks
# x_chunk = x[:, :, ix_start:ix_end]
# y_chunk = y[ip_start:ip_end, :, :]
# z_chunk = z[:, :, :]
x_chunk = x
y_chunk = y
z_chunk = z

y_offset = y_chunk.min()
x_offset = x_chunk.min()

In [17]:
# ###################################################
# identify candidate beads
# ###################################################

dz_min, dxy_min = min_spot_sep
min_separations = (dz_min, dxy_min, dxy_min)

footprint = localize.get_max_filter_footprint(min_separations, drs)
centers_guess_inds, amps = localize.find_peak_candidates(imgs_filtered, footprint, thresholds[ch])

# convert to xyz coordinates
centers_guess = np.stack((z_chunk[centers_guess_inds[:, 0], 0, 0],
                            y_chunk[0, centers_guess_inds[:, 1], 0],
                            x_chunk[0, 0, centers_guess_inds[:, 2]]),
                            axis=1)
# /!\ check here the order for z/y/x_chunk wrt centers_guess_inds

In [18]:
if len(centers_guess) != 0:
    # ###################################################
    # average multiple points too close together. Necessary bc if naive threshold, may identify several points
    # from same spot. Particularly important if spots have very different brightness levels.
    # ###################################################
    tstart = time.perf_counter()

    inds = np.ravel_multi_index(centers_guess_inds.transpose(), imgs_filtered.shape)
    weights = imgs_filtered.ravel()[inds]
    centers_guess, inds_comb = localize.filter_nearby_peaks(centers_guess, dxy_min, dz_min, weights=weights,
                                                            mode="average")

    amps = amps[inds_comb]
    print("Found %d points separated by dxy > %0.5g and dz > %0.5g in %0.1fs" %
            (len(centers_guess), dxy_min, dz_min, time.perf_counter() - tstart))

   

Found 144 points separated by dxy > 0.486 and dz > 2.2268 in 0.1s


In [35]:
np.array(coords_roi).shape

In [24]:
# ###################################################
# prepare ROIs
# ###################################################
tstart_prep_roi = time.perf_counter()

# cut rois out
roi_size = roi_fns.get_roi_size(roi_size, dc, dstage, ensure_odd=True)
rois, img_rois, coords_roi = zip(*[localize.get_roi(c, imgs_chunk, (z_chunk, y_chunk, x_chunk), roi_size) for c in centers_guess])
# xrois, yrois, zrois = coords_roi
rois = np.asarray(rois)

# exclude some regions of roi
roi_masks = [localize.get_roi_mask(c, (np.inf, 0.5 * roi_size[1]), (zrois[ii], yrois[ii], xrois[ii])) for
                ii, c in enumerate(centers_guess)]

# mask regions
xrois, yrois, zrois, img_rois = zip(
    *[(np.expand_dims(xr[rm], axis=0),
        np.expand_dims(yr[rm], axis=0),
        np.expand_dims(zr[rm], axis=0),
        np.expand_dims(ir[rm], axis=0))
        for xr, yr, zr, ir, rm in zip(xrois, yrois, zrois, img_rois, roi_masks)])

# extract guess values
with np.errstate(invalid="ignore"):
    bgs = np.array([np.mean(r) for r in img_rois])
    # why this euclidean distance or std? ---v
    sxs = np.array([np.sqrt(np.sum(ir * (xr - cg[2]) ** 2) / np.sum(ir)) for ir, xr, cg in
                    zip(img_rois, xrois, centers_guess)])
    sys = np.array([np.sqrt(np.sum(ir * (yr - cg[1]) ** 2) / np.sum(ir)) for ir, yr, cg in
                    zip(img_rois, yrois, centers_guess)])
    sxys = 0.5 * (sxs + sys)
    szs = np.array([np.sqrt(np.sum(ir * (zr - cg[0]) ** 2) / np.sum(ir)) for ir, zr, cg in
                    zip(img_rois, zrois, centers_guess)])

# get initial parameter guesses
init_params = np.stack((amps, centers_guess[:, 2],
                centers_guess[:, 1],
                centers_guess[:, 0],
                sxys, szs, bgs), axis=1)

print("Prepared %d rois and estimated initial parameters in %0.2fs" % (len(rois), time.perf_counter() - tstart_prep_roi))

AttributeError: module 'localize_psf.localize' has no attribute 'get_roi_mask'

In [ ]:
# # ###################################################
# # localization
# # ###################################################
# print("Starting localization fitting for %d rois" % centers_guess.shape[0])
# tstart = time.perf_counter()

# fit_params, fit_states, chi_sqrs, niters, fit_t = localize.fit_gauss_rois(img_rois, (zrois, yrois, xrois),
#                                                                             init_params, estimator="LSE",
#                                                                             model="gaussian",
#                                                                             sf=1, dc=dc, angles=(0., theta, 0.))

# tend = time.perf_counter()
# print("Localization fitting for %d rois took %0.2fs" % (len(rois), tend - tstart))

# # fitting
# fit_results = np.stack((fit_states, chi_sqrs, niters), axis=1)

# # ###################################################
# # preliminary fitting of results
# # ###################################################
# tstart = time.perf_counter()

# to_keep, conditions, condition_names, filter_settings = localize_skewed.filter_localizations(
#     fit_params, init_params, (z_chunk, y_chunk, x_chunk),
#     (sigma_z, 3*sigma_xy), min_spot_sep,
#     (sigmas_min, sigmas_max),
#     fit_thresholds[ch],
#     dist_boundary_min=(0.5 * sigma_z, sigma_xy))

# print("Identified %d/%d plausible localizations in %0.1fs" % (np.sum(to_keep), to_keep.size, time.perf_counter() - tstart))

# # ###################################################
# # correct ROIs for full volume
# # ###################################################
# rois[:, :2] += ip_start
# rois[:, 4:] += ix_start

## Use Peter's pipeline

In [9]:
from importlib import reload
reload(localize)

<module 'localize_psf.localize' from '/home/alexis/Documents/Pro/Postdoc_ASU/Projects/localize-psf/localize_psf/localize.py'>

In [12]:
dxy = dc
dz = dstage
threshold = thresholds[ch]
sigma_bounds = (sigmas_min, sigmas_max)
fit_amp_min = 60 #fit_thresholds[ch]

(z, y, x), fit_params, init_params, rois = localize.localize_spots(
    imgs, dxy, dz, threshold, roi_size, 
    filter_sigma_small, filter_sigma_large,
    min_spot_sep, verbose=True,
    )

filtered image in 3.74s
identified 380 candidates in 0.19s
Found 317 points separated by dxy > 0.486 and dz > 2.2268 in 0.0s
Prepared 317 rois and estimated initial parameters in 0.29s
starting fitting for 317 rois


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   25.3s


Localization took 38.63s


[Parallel(n_jobs=-1)]: Done 317 out of 317 | elapsed:   38.6s finished


In [13]:
fit_params.shape

(317, 7)

In [14]:
amplitude, center_x, center_y, center_z, sigma_xy, sigma_z, offset = [fit_params[:, i] for i in range(fit_params.shape[1])]

In [15]:
center_fit = np.vstack([center_z, center_y, center_x]).T
center_fit = center_fit / np.array([dstage, dc, dc])
# spots_filtered = spots_guessed[to_keep, :]

# invert z orientation
# spots_guessed[:, 0] = imgs.shape[0] - spots_guessed[:, 0]

In [16]:
# viewer = napari.Viewer()
# viewer.add_image(imgs, name='img')
# viewer.add_image(img_filtered, name='img_filtered')
# viewer.add_image(im_fitted, name='im_fitted')
viewer.add_points(center_fit, name='center_fit', blending='additive', size=3, face_color='b')
# viewer.add_points(spots_filtered, name='spots_filtered', blending='additive', size=3, face_color='g')

<Points layer 'center_fit' at 0x7fbb14340070>

that works, let's reproduce the successive steps of this function

## Successive steps of Peter's pipeline

In [17]:
# localize_spots(imgs, dxy, dz, threshold, roi_size, filter_sigma_small, filter_sigma_large,
#                    min_spot_sep, use_gpu_fit=GPUFIT_AVAILABLE, use_gpu_filter=CUPY_AVAILABLE, verbose=True)

dxy = dc
dz = dstage
threshold = thresholds[ch]
sigma_bounds = (sigmas_min, sigmas_max)
fit_amp_min = 60 #fit_thresholds[ch]

use_gpu_fit=False
use_gpu_filter=False
verbose=True

In [18]:
# make sure inputs correct types
roi_size = np.array(roi_size, copy=True)
min_spot_sep = np.array(min_spot_sep, copy=True)
filter_sigma_large = np.array(filter_sigma_large, copy=True)
filter_sigma_small = np.array(filter_sigma_small, copy=True)

# check if is 2D
if imgs.ndim == 2:
    imgs = np.expand_dims(imgs, axis=0)

data_is_2d = imgs.shape[0] == 1

if data_is_2d:
    roi_size[0] = 0
    filter_sigma_large[0] = 0
    filter_sigma_small[0] = 0
    min_spot_sep[0] = 0

# unpack arguments
z, y, x = localize.get_coords(imgs.shape, (dz, dxy, dxy))
dz_min_sep, dxy_min_sep = min_spot_sep
roi_size_pix = roi_fns.get_roi_size(roi_size, dxy, dz, ensure_odd=True)

# ###################################
# filter images
# ###################################
tstart = time.perf_counter()

ks = localize.get_filter_kernel(filter_sigma_small, (dz, dxy, dxy))
kl = localize.get_filter_kernel(filter_sigma_large, (dz, dxy, dxy))
imgs_hp = localize.filter_convolve(imgs, ks, use_gpu=use_gpu_filter)
imgs_lp = localize.filter_convolve(imgs, kl, use_gpu=use_gpu_filter)
imgs_filtered = imgs_hp - imgs_lp

if verbose:
    print("filtered image in %0.2fs" % (time.perf_counter() - tstart))

filtered image in 4.21s


In [19]:
viewer.add_image(imgs_filtered, name='img_filtered')

<Image layer 'img_filtered' at 0x7fbb17668580>

In [20]:
# ###################################
# identify candidate beads
# ###################################
tstart = time.perf_counter()

footprint = localize.get_max_filter_footprint((dz_min_sep, dxy_min_sep, dxy_min_sep), (dz, dxy, dxy))
centers_guess_inds, amps = localize.find_peak_candidates(imgs_filtered, footprint, threshold)

# real coordinates
centers_guess = np.stack((z[centers_guess_inds[:, 0], 0, 0],
                            y[0, centers_guess_inds[:, 1], 0],
                            x[0, 0, centers_guess_inds[:, 2]]), axis=1)

if verbose:
    print("identified %d candidates in %0.2fs" % (len(centers_guess_inds), time.perf_counter() - tstart))

identified 380 candidates in 0.31s


In [21]:
spots_guessed = centers_guess / np.array([dstage, dc, dc])
viewer.add_points(spots_guessed, name='spots_guessed', blending='additive', size=3, face_color='r')

<Points layer 'spots_guessed' at 0x7fbb176685b0>

In [22]:
# ###################################
# identify candidate beads
# ###################################
if len(centers_guess_inds) == 0:
    # return None
    pass
else:
    # ###################################################
    # average multiple points too close together. Necessary bc if naive threshold, may identify several points
    # from same spot. Particularly important if spots have very different brightness levels.
    # ###################################################
    tstart = time.perf_counter()

    inds = np.ravel_multi_index(centers_guess_inds.transpose(), imgs_filtered.shape)
    weights = imgs_filtered.ravel()[inds]
    centers_guess, inds_comb = localize.filter_nearby_peaks(centers_guess, dxy_min_sep, dz_min_sep,
                                                    weights=weights, mode="average")

    amps = amps[inds_comb]
    if verbose:
        print("Found %d points separated by dxy > %0.5g and dz > %0.5g in %0.1fs" %
                (len(centers_guess), dxy_min_sep, dz_min_sep, time.perf_counter() - tstart))

    # ###################################################
    # prepare ROIs
    # ###################################################
    tstart = time.perf_counter()

    rois, img_rois, coords = zip(*[localize.get_roi(c, imgs, (z, y, x), roi_size_pix) for c in centers_guess])
    zrois, yrois, xrois = zip(*coords)
    rois = np.asarray(rois)

    # extract guess values
    bgs = np.array([np.mean(r) for r in img_rois])
    sxs = np.array([np.sqrt(np.sum(ir * (xr - cg[2]) ** 2) / np.sum(ir)) for ir, xr, cg in
                    zip(img_rois, xrois, centers_guess)])
    sys = np.array([np.sqrt(np.sum(ir * (yr - cg[1]) ** 2) / np.sum(ir)) for ir, yr, cg in
                    zip(img_rois, yrois, centers_guess)])
    sxys = 0.5 * (sxs + sys)

    if data_is_2d:
        cz_guess = np.zeros(len(img_rois))
        szs = np.ones(len(img_rois))
    else:
        cz_guess = centers_guess[:, 0]
        szs = np.array([np.sqrt(np.sum(ir * (zr - cg[0]) ** 2) / np.sum(ir)) for ir, zr, cg in
                        zip(img_rois, zrois, centers_guess)])

    # get initial parameter guesses
    init_params = np.stack((amps,
                            centers_guess[:, 2], centers_guess[:, 1], cz_guess,
                            sxys, szs, bgs), axis=1)

    if verbose:
        print("Prepared %d rois and estimated initial parameters in %0.2fs" %
                (len(rois), time.perf_counter() - tstart))

Found 317 points separated by dxy > 0.486 and dz > 2.2268 in 0.1s
Prepared 317 rois and estimated initial parameters in 0.39s


In [23]:
spots_guessed = centers_guess / np.array([dstage, dc, dc])
viewer.add_points(spots_guessed, name='spots_guessed_merged', blending='additive', size=3, face_color='g')

<Points layer 'spots_guessed_merged' at 0x7fbb42800af0>

In [24]:
# In the conditional loop:

# ###################################################
# localization
# ###################################################
if verbose:
    print("starting fitting for %d rois" % centers_guess.shape[0])
tstart = time.perf_counter()

fixed_params = [False] * 7

# if 2D, don't want to fit cz or sz
if data_is_2d:
    fixed_params[5] = True
    fixed_params[3] = True

fit_params, fit_states, chi_sqrs, niters, fit_t = localize.fit_gauss_rois(img_rois, (zrois, yrois, xrois),
                                                                    init_params, estimator="LSE",
                                                                    sf=1, dc=dxy, angles=(0., 0., 0.),
                                                                    fixed_params=fixed_params,
                                                                    use_gpu=use_gpu_fit)

tend = time.perf_counter()
if verbose:
    print("Localization took %0.2fs" % (tend - tstart))

# # ###################################################
# # filter fits
# # ###################################################
# to_keep, conditions, condition_names, filter_settings = \
#     filter_localizations(fit_params, init_params, (z, y, x), fit_dist_max_err, min_spot_sep,
#                          sigma_bounds, fit_amp_min, dist_boundary_min)

# if verbose:
#     print("Identified %d likely candidates" % np.sum(to_keep))

# return (z, y, x), fit_params, init_params, rois, to_keep, conditions, condition_names, filter_settings
# return (z, y, x), fit_params, init_params, rois

starting fitting for 317 rois


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done 224 tasks      | elapsed:   29.0s


Localization took 41.67s


[Parallel(n_jobs=-1)]: Done 317 out of 317 | elapsed:   41.7s finished


In [26]:
center_fit = np.array([fit_params[:, 3], fit_params[:, 2], fit_params[:, 1]]).T
# center_fit = np.array([fit_params[:, 1], fit_params[:, 2], fit_params[:, 3]]).T
center_fit = center_fit / np.array([dstage, dc, dc])
viewer.add_points(center_fit, name='center_fit succ', blending='additive', size=3, face_color='b')

<Points layer 'center_fit succ' at 0x7fbb4280b2e0>

In [27]:
center_fit.shape

(317, 3)

In [28]:
# comparison with localize_spots coordinates
center_fit = np.vstack([center_z, center_y, center_x]).T
center_fit = center_fit / np.array([dstage, dc, dc])
viewer.add_points(center_fit, name='localize_spots', blending='additive', size=3, face_color='r')

<Points layer 'localize_spots' at 0x7fbb41bad9a0>